In [1]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("Solarize_Light2")

%matplotlib inline

In [7]:
telemetry_df = pd.read_csv("./csv/maq1.csv", sep = ';')
errors_df = pd.read_csv("./csv/errosmaq1.csv")
maq1erros = pd.read_csv("./csv/maq1erros.csv", sep = ';')

In [8]:
telemetry_df['datetime'] = pd.to_datetime(telemetry_df['datetime'])

In [9]:
errors_df['datetime'] = pd.to_datetime(errors_df['datetime'])

In [10]:
maq1erros

,datetime,machineID,volt,rotate,pressure,vibration,error
0,01/01/2015 06:00,1,176.217853,418.504078,113.077935,45.087686,0
1,01/01/2015 07:00,1,162.879223,402.747490,95.460525,43.413973,0
2,01/01/2015 08:00,1,170.989902,527.349825,75.237905,34.178847,0
3,01/01/2015 09:00,1,162.462833,346.149335,109.248561,41.122144,0
4,01/01/2015 10:00,1,157.610021,435.376873,111.886648,25.990511,0
...,...,...,...,...,...,...,...
8756,01/01/2016 02:00,1,153.724780,450.254337,85.197443,43.698137,0
8757,01/01/2016 03:00,1,172.418099,375.701907,95.568265,42.111661,0
8758,01/01/2016 04:00,1,174.588145,460.505353,95.483913,33.293572,0
8759,01/01/2016 05:00,1,184.224228,438.823315,102.694358,46.609515,0


In [11]:
maq1erros['datetime'] = pd.to_datetime(maq1erros['datetime'])

In [12]:
pip install h2o==3.32.1.6

Note: you may need to restart the kernel to use updated packages.


In [13]:
import h2o

In [14]:
h2o.init(ip="172.16.131.140", port="54321")

Checking whether there is an H2O instance running at http://172.16.131.140:54321 . connected.


H2O_cluster_uptime:,13 days 0 hours 41 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,5 months and 5 days !!!
H2O_cluster_name:,H2O_CLUSTER
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,31.45 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,32
H2O_cluster_status:,"locked, healthy"


In [15]:
pip install cane

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [17]:

import h2o
from h2o.automl import H2OAutoML, get_leaderboard
import mlflow
import mlflow.h2o
from mlflow.tracking import MlflowClient
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_absolute_error as MAE

In [18]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [19]:
import boto3
import os

In [20]:
os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow:5000"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://minio:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

In [21]:
from sklearn import metrics
from sklearn.metrics import accuracy_score as AUC
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve

In [23]:
train_h2o=h2o.H2OFrame(train)
test_h2o=h2o.H2OFrame(test)

print (train_h2o)

x = train_h2o.columns
y = target
x.remove(y)

print (x)
print (y)

train_h2o[y] = train_h2o[y].asfactor()
test_h2o[y] = test_h2o[y].asfactor()

experiment_name = 'provatelemetria'

client = MlflowClient()

try:
    experiment = mlflow.create_experiment(experiment_name)
except:
    experiment = client.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_name)


with mlflow.start_run():
    model = H2OAutoML(max_runtime_secs = 600, seed = 1, project_name = "provatelemetria", nfolds=10, sort_metric = 'AUC')
    model.train(x=x, y=y, training_frame = train_h2o)
   
    mlflow.log_param("runtime_sec", model.max_runtime_secs)
    mlflow.log_param("seed", model.seed)
    mlflow.log_param("nfolds", model.nfolds)
    mlflow.h2o.log_model(model.leader, "model")
    
    lb = model.leaderboard
    lb = get_leaderboard(model, extra_columns='ALL')
    print('')
    print(lb.head(rows=lb.nrows))
   # model.leader.varimp_plot()
    pred = model.predict(test_h2o)
    print(pred.head())
    pred_df = h2o.as_list(pred)

    perf = model.leader.model_performance(test_h2o)
    print(perf)
    print(test[y])
 #   auc = AUC(test[y], pred)
 #   precision, recall, thresholds = precision_recall_curve(test, pred_df)
  #  auc_precision_recall = auc(recall, precision)
  #  print(auc_precision_recall)
   # nmae = NMAE(mae, test[y])
  #  print('--------AUC-------', auc)
   # print('--------NMAE-------', nmae)
   # mlflow.log_metric("auc", auc)
    #print('--------AUC-------', AUC)
   # mlflow.log_metric("nmae", nmae)
    #exm = model.leader.explain_row(test_h2o, row_index=200)

#all_mlflow_runs = client.list_run_infos(experiment.experiment_id)
#if len(all_mlflow_runs) > 0:
#    run_info = all_mlflow_runs[-1]
#    model = mlflow.h2o.load_model("mlruns/{exp_id}/{run_id}/artifacts/model/".format(exp_id=experiment.experiment_id,run_id=run_info.run_uuid))
#    result = model.predict(test_h2o)
#else:
#    raise Exception('Run the training first')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


datetime,machineID,volt,rotate,pressure,vibration,error
2015-01-01 06:00:00,1,176.218,418.504,113.078,45.0877,0
2015-01-01 07:00:00,1,162.879,402.747,95.4605,43.414,0
2015-01-01 08:00:00,1,170.99,527.35,75.2379,34.1788,0
2015-01-01 09:00:00,1,162.463,346.149,109.249,41.1221,0
2015-01-01 10:00:00,1,157.61,435.377,111.887,25.9905,0
2015-01-01 11:00:00,1,172.505,430.323,95.927,35.655,0
2015-01-01 12:00:00,1,156.556,499.072,111.756,42.7539,0
2015-01-01 13:00:00,1,172.523,409.625,101.001,35.482,0
2015-01-01 14:00:00,1,175.325,398.649,110.624,45.4823,0
2015-01-01 15:00:00,1,169.218,460.851,104.848,39.9017,0



['datetime', 'machineID', 'volt', 'rotate', 'pressure', 'vibration']
error
AutoML progress: |████████████████████████████████████████████████████████| 100%



model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
GBM_grid__1_AutoML_20220125_163329_model_61,0.659549,0.0744092,0.00749749,0.471151,0.0799817,0.00639707,1067,0.035297,GBM
GBM_grid__1_AutoML_20220125_163329_model_15,0.639986,0.0851937,0.00817712,0.434654,0.0791565,0.00626575,2638,0.058652,GBM
GBM_grid__1_AutoML_20220125_163329_model_22,0.627816,0.103035,0.00538687,0.351454,0.0869884,0.00756698,1628,0.059797,GBM
GBM_grid__1_AutoML_20220125_163329_model_62,0.627736,0.0918014,0.00548617,0.337319,0.080068,0.00641088,2433,0.082255,GBM
GBM_grid__1_AutoML_20220125_163329_model_44,0.616255,0.0404362,0.00561899,0.354716,0.0703399,0.0049477,661,0.022175,GBM
GBM_grid__1_AutoML_20220125_163329_model_47,0.60777,0.0856271,0.00575357,0.464064,0.0818775,0.00670392,2102,0.060994,GBM
StackedEnsemble_BestOfFamily_AutoML_20220125_163329,0.605349,0.0267283,0.0058235,0.389066,0.0643236,0.00413752,640,0.035563,StackedEnsemble
GBM_grid__1_AutoML_20220125_163329_model_28,0.599982,0.0366006,0.0053509,0.470012,0.0700229,0.0049032,581,0.018073,GBM
GBM_grid__1_AutoML_20220125_163329_model_77,0.598128,0.0423394,0.00500554,0.356939,0.0703579,0.00495024,764,0.025107,GBM
GBM_grid__1_AutoML_20220125_163329_model_48,0.594858,0.141139,0.00625028,0.483799,0.0939721,0.00883076,2260,0.069828,GBM



gbm prediction progress: |████████████████████████████████████████████████| 100%


...


Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,3.426620e-06,0.004963,345.0
1,max f2,3.426620e-06,0.012315,345.0
2,max f0point5,3.426620e-06,0.003108,345.0
3,max accuracy,9.999908e-01,0.997346,0.0
4,max precision,3.426620e-06,0.002488,345.0
5,max recall,3.426620e-06,1.000000,345.0
6,max specificity,9.999908e-01,0.999557,0.0
7,max absolute_mcc,3.455256e-06,0.035411,278.0
8,max min_per_class_accuracy,3.454865e-06,0.416223,283.0
9,max mean_per_class_accuracy,3.426620e-06,0.555629,345.0



Gains/Lift Table: Avg response rate:  0.22 %, avg score:  0.13 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010172,7.691884e-06,0.000000,0.000000,0.000000,0.130733,0.000000,0.130733,0.0,0.0,-100.000000,-100.000000,-0.010195
1,2,0.020345,5.366819e-06,0.000000,0.000000,0.000000,0.000006,0.000000,0.065370,0.0,0.0,-100.000000,-100.000000,-0.020390
2,3,0.030075,5.154219e-06,0.000000,0.000000,0.000000,0.000005,0.000000,0.044222,0.0,0.0,-100.000000,-100.000000,-0.030142
3,4,0.040248,4.285881e-06,0.000000,0.000000,0.000000,0.000005,0.000000,0.033046,0.0,0.0,-100.000000,-100.000000,-0.040337
4,5,0.050420,4.222182e-06,0.000000,0.000000,0.000000,0.000004,0.000000,0.026380,0.0,0.0,-100.000000,-100.000000,-0.050532
5,6,0.100398,4.072711e-06,0.000000,0.000000,0.000000,0.000004,0.000000,0.013250,0.0,0.0,-100.000000,-100.000000,-0.100621
6,7,0.150376,3.505668e-06,0.000000,0.000000,0.000000,0.000004,0.000000,0.008848,0.0,0.0,-100.000000,-100.000000,-0.150709
7,8,0.200354,3.463758e-06,0.000000,0.000000,0.000000,0.000003,0.000000,0.006642,0.0,0.0,-100.000000,-100.000000,-0.200798
8,9,0.300310,3.455850e-06,0.000000,0.000000,0.000000,0.000003,0.000000,0.004432,0.0,0.0,-100.000000,-100.000000,-0.300975
9,10,0.400265,3.455095e-06,2.000885,0.499669,0.004425,0.000003,0.001105,0.003326,0.2,0.2,100.088496,-50.033149,-0.200709




6500    0
6501    0
6502    0
6503    0
6504    0
       ..
8756    0
8757    0
8758    0
8759    0
8760    0
Name: error, Length: 2261, dtype: int64


In [22]:
target = 'error'
#train, test = train_test_split(dfconceito, test_size=0.2, random_state=25)
train = maq1erros[0:6500]
test = maq1erros[6500:8761]